# CBAMResidualDynamicSA — Detaylı Notlar

Bu defter yalnızca aşağıdaki modülü anlatır:

- `CBAMResidualDynamicSA`

Bu sınıfın görevi:
1. **Channel Attention** (CA) uygula  
2. Çıkan sonuç üzerine **Dynamic Spatial Attention** (SA) uygula  
3. İsteğe bağlı olarak **residual karışım** ile girişe geri bağla  
4. İsteğe bağlı olarak debug için **haritaları/ağırlıkları** geri döndür

> Not: CA ve SA’nın iç yapısı ayrı notlarda detaylandırıldı. Burada bu ikisini **nasıl birleştirdiğimiz** ve residual/alpha mantığı anlatılıyor.


## 1) Sınıfın Büyük Resmi

Giriş: `x`  → shape: **(B, C, H, W)**

Aşamalar:

1) `y = CA(x)`  
- Kanal bazlı “hangi kanal önemli?” ölçekleme

2) `y = SA(y)`  
- Uzamsal “hangi konum önemli?” maskeleme

3) (Opsiyonel) residual karışım:  
- `out = x + alpha * (y - x)`  
- Bu ifade pratikte şuna eşittir:  
  - `out = (1 - alpha) * x + alpha * y`

Bu sayede `alpha`:
- `0` ise: **çıktı = x** (attention etkisi yok)
- `1` ise: **çıktı = y** (tam attention)
- aradaysa: **yumuşak karışım** (stabil/kolay öğrenme)


## 2) Parametreler — Ne İşe Yarıyor?

### Temel
- **channels**: Girişin kanal sayısı (C). CA bunu bilmek zorunda; SA zaten (avg/max) ile 2 kanala indirger.
- **reduction, min_hidden**: Channel Attention içindeki MLP’nin “daraltma” ayarları.
- **return_maps**: Debug modu. True ise forward daha fazla çıktı döndürür:
  - CA map (`ca`)
  - SA map (`sa`)
  - CA fusion ağırlıkları (`fusion_w`)
  - SA router ağırlıkları (`router_w`)

### CA (ChannelAttentionFusionT) ile ilgili
- **ca_fusion**: avg ve max squeeze çıktıları nasıl birleştirilecek? `"sum"` veya `"softmax"`.
- **ca_gate**: Channel maskesi üretirken `sigmoid` mi `hardsigmoid` mi?
- **ca_temperature**: Channel maskesinin keskinlik/yumuşaklık kontrolü.
- **ca_act**: CA MLP içindeki aktivasyon (`relu` / `silu`).

### SA (DynamicSpatialAttention) ile ilgili
- **sa_kernels**: Normal branch kernel listesi (örn `(3,7)`).
- **sa_use_dilated**: Ek dilated branch eklensin mi?
- **sa_dilated_kernel, sa_dilated_d**: Dilated branch ayarları.
- **sa_router_hidden**: Router’ın ara kanal sayısı (karar kapasitesi).
- **sa_gate, sa_temperature**: Spatial mask için gate ve temperature.

### Ortak Temperature Öğrenilebilirliği
- **learnable_temperature**: True ise hem CA hem SA içindeki `T` öğrenilebilir olur.

### Residual ve Alpha
- **residual**: True ise `out = x + alpha*(y-x)` yapılır. False ise direkt `out=y`.
- **alpha_init**: alpha başlangıç değeri.
- **learnable_alpha**:
  - True → alpha öğrenilir (`nn.Parameter`)
  - False → alpha sabit kalır (`register_buffer`)


## 3) `alpha` Neden Var?

Attention bloklarında bazen problem şudur:
- CA+SA güçlü bir maske üretir
- Modelin ilk dönemlerinde bu maske fazla agresif olabilir
- Gradyan/optimizasyon daha zor hale gelebilir

`alpha` ile amaç:
- attention’ı **kontrollü** devreye almak
- “tam bas” yerine “kademeli karıştır” mantığı

Matematik:
- `out = x + alpha*(y-x)`  
- `out = (1-alpha)*x + alpha*y`

Bu, residual öğrenmeyi genelde daha stabil yapar.


## 4) `nn.Parameter` vs `register_buffer` — Neden İkisi Var?

Alpha iki şekilde tutulabilir:

### a) Öğrenilebilir alpha
```python
self.alpha = nn.Parameter(torch.tensor(alpha_init))
```
- optimizer alpha’yı günceller
- model “ne kadar residual karışım yapacağını” öğrenir

### b) Sabit alpha
```python
self.register_buffer("alpha", torch.tensor(alpha_init))
```
- alpha öğrenilmez ama:
  - model ile birlikte GPU’ya taşınır
  - checkpoint içine girer
- eğitim boyunca sabit kalır


In [ ]:
import torch
import torch.nn as nn

# Not: ChannelAttentionFusionT ve DynamicSpatialAttention
# sınıflarının aynı dosyada/ortamda tanımlı olduğu varsayılır.
# Bu notebook, sadece CBAMResidualDynamicSA'nın mantığına odaklanır.


class CBAMResidualDynamicSA(nn.Module):
    def __init__(
        self,
        channels: int,
        reduction: int = 16,
        min_hidden: int = 4,
        ca_fusion: str = "softmax",
        ca_gate: str = "sigmoid",
        ca_temperature: float = 1.0,
        ca_act: str = "relu",
        sa_gate: str = "sigmoid",
        sa_temperature: float = 1.0,
        learnable_temperature: bool = False,
        sa_kernels=(3, 7),
        sa_use_dilated: bool = True,
        sa_dilated_kernel: int = 7,
        sa_dilated_d: int = 2,
        sa_router_hidden: int = 8,
        residual: bool = True,
        alpha_init: float = 1.0,
        learnable_alpha: bool = False,
        return_maps: bool = False,
    ):
        super().__init__()
        self.return_maps = return_maps
        self.residual = residual

        self.ca = ChannelAttentionFusionT(
            channels=channels,
            reduction=reduction,
            min_hidden=min_hidden,
            fusion=ca_fusion,
            gate=ca_gate,
            temperature=ca_temperature,
            learnable_temperature=learnable_temperature,
            act=ca_act,
            return_fusion_weights=return_maps,
        )

        self.sa = DynamicSpatialAttention(
            kernels=sa_kernels,
            use_dilated=sa_use_dilated,
            dilated_kernel=sa_dilated_kernel,
            dilated_d=sa_dilated_d,
            gate=sa_gate,
            temperature=sa_temperature,
            learnable_temperature=learnable_temperature,
            router_hidden=sa_router_hidden,
            return_router_weights=return_maps,
        )

    # alpha_init, residual karışımın BAŞLANGIÇTA ne kadar güçlü olacağını belirleyen ilk değerdir.
    # alpha_init, alpha’nın ilk değeri:: Model daha eğitime başlamadan önce, attention’ın etkisi yüzde kaç olsun?
        if residual:
            if learnable_alpha:
                self.alpha = nn.Parameter(torch.tensor(float(alpha_init)))
            else:
                self.register_buffer("alpha", torch.tensor(float(alpha_init)))

    def forward(self, x: torch.Tensor):
        # return_maps şu demek: “Sadece sonucu mu döndüreyim, yoksa attention maskelerini/ağırlıklarını da yanında vereyim mi?”

        # return_maps=True → CA 3 şey döndürür :: :: return_maps=False → CA 2 şey döndürür
        if self.return_maps:
            # ca :: Maske 
            # y :: Attention uygulanmış çıktı
            # fusion_w :: avg mi öenmli max mı ? 
            # router_w :: Spatial de olan işlem aslında.Yani oluşan kernellara güvenden çıkan değerler.Düz tabirle ise :: SA’nın “hangi kernel branch daha işe yarıyor?” kararının ağırlıklarıdır.
            # out :: Nihai çıktı
            y, ca, fusion_w = self.ca(x)
            y, sa, router_w = self.sa(y)
            out = x + self.alpha * (y - x) if self.residual else y
            # y - x = attention’ın x’i ne kadar değiştirdiği
            # alpha = bu değişikliğin ne kadarını ekleyelim? :: alpha = “attention’a ne kadar katayım?” katsayısı
            return out, ca, sa, fusion_w, router_w
        
# return_maps bu blokta “debug modu” gibi çalışır. Amaç, sadece nihai çıktıyı vermek mi, yoksa attention’ın ürettiği maskeleri ve ağırlıkları da yanında göstermek mi kararını vermektir.

# Eğer self.return_maps True ise, model önce self.ca(x) çağrısı ile channel attention uygular ve bu çağrıdan üç şey alır: attention uygulanmış ara çıktı y, kanal maskesi ca 
# ve (avg mi max mı daha baskın?) bilgisini veren fusion_w. Ardından bu y çıktısını self.sa(y) içine sokar; 
# burada da spatial attention uygulanır ve yine üç şey alınır: yeni y, uzamsal maske sa ve branch seçim ağırlıkları router_w.
    
# Daha sonra residual açıksa x ile y arasında alpha katsayısı ile karışım yapılır (out = x + alpha*(y-x)), residual kapalıysa doğrudan y kullanılır.
# Bu debug modunda fonksiyon beş değer döndürür: out, ca, sa, fusion_w, router_w. 
# Yani hem nihai sonucu hem de “model nereye bakıyor/neyi seçiyor?” diye inceleyebileceğin tüm haritaları ve ağırlıkları verir.

# Eğer self.return_maps False ise, aynı attention işlemleri yine yapılır; fakat bu sefer self.ca(x) ve self.sa(y) çağrılarından dönen maskeler _ ile alınmayıp bilerek atılır.
# Yani model channel ve spatial maskeleri hesaplayıp uygular, ama senin dışarıda görmene gerek olmadığı için sadece attention uygulanmış y üzerinden devam eder. 
# Ardından yine residual karışımı hesaplanır ve fonksiyon yalnızca tek çıktı döndürür: out. Bu mod “normal kullanım” modudur; maskeler içeride uygulanır, dışarıya yalnızca nihai çıktı verilir.

        y, _ = self.ca(x)
        y, _ = self.sa(y)
        out = x + self.alpha * (y - x) if self.residual else y
        return out


## 5) Forward Akışı — Satır Satır (Önemli Kısım)

Aşağıdaki anlatım, `forward()` içindeki iki akışı da kapsar.

---

### A) `return_maps = True` iken (debug modu)

```python
y, ca, fusion_w = self.ca(x)
y, sa, router_w = self.sa(y)
out = x + alpha * (y - x) if residual else y
return out, ca, sa, fusion_w, router_w
```

- **CA** üç şey döndürür:
  1) `y`: attention uygulanmış feature
  2) `ca`: channel mask (B,C,1,1)
  3) `fusion_w`: avg/max fuse ağırlıkları (2,) (sadece fusion="softmax" iken anlamlı)

- **SA** üç şey döndürür:
  1) `y`: spatial attention uygulanmış feature
  2) `sa`: spatial mask (B,1,H,W)
  3) `router_w`: branch seçim ağırlıkları (B,K)

- Sonra **residual** açıksa:
  - `out = x + alpha*(y - x)`  
  - yani `x` ile `y` arasında kontrollü karışım

Debug çıktıları ne işe yarar?
- `ca` ve `sa` görselleştirilebilir
- `fusion_w` ile CA’da avg mı max mı baskın takip edilir
- `router_w` ile SA’da hangi branch’in baskın olduğu takip edilir

---

### B) `return_maps = False` iken (normal mod)

```python
y, _ = self.ca(x)
y, _ = self.sa(y)
out = x + alpha*(y-x) if residual else y
return out
```

Bu modda:
- Ek harita/weight dönmez
- Daha hafif ve üretime daha uygun

---

## 6) Residual Kapalıysa Ne Olur?

```python
out = y
```

- Bu durumda CA+SA çıktısı direkt alınır.
- Alpha zaten oluşturulmuş olsa bile (kodda residual False ise kullanılmaz).

---

## 7) `alpha` ile “Karışım” Mantığı

`out = x + alpha*(y-x)` ifadesi şunu sağlar:

- `alpha = 0` → `out = x`
- `alpha = 1` → `out = y`
- `alpha = 0.5` → `out = 0.5*x + 0.5*y`

Bu sayede:
- attention etkisini kontrollü şekilde açıp kapatabilirsin
- stabilite artabilir (özellikle çok güçlü maskelerde)


## 8) Mini Notlar (Sık Takılınan Yerler)

- **Neden CA önce, SA sonra?**  
  CBAM geleneği: önce kanal seçimi, sonra uzamsal odak.  
  Kanal seçiminden sonra uzamsal maske daha anlamlı çalışır.

- **CA ve SA ikisi de mask üretiyor, neden ikisi birden?**  
  CA: “hangi feature kanalları önemli?”  
  SA: “hangi konum önemli?”  
  İkisi farklı eksenlerde çalışır, birbirini tamamlar.

- **return_maps neden hem CA’ya hem SA’ya gidiyor?**  
  Çünkü debug çıktıları ayrı ayrı izlenmek istenir:
  - CA: `fusion_w`
  - SA: `router_w`

- **learnable_temperature tek parametreyle ikisini de etkiliyor**  
  Bu tasarım seçimi: tek flag ile CA+SA sıcaklıklarının öğrenilebilirliği açılır.
